In [1]:
import torch
print("La version de torch est : ", torch.__version__)
print("Le calcul GPU est disponible ? ", torch.cuda.is_available())
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

La version de torch est :  2.6.0+cu124
Le calcul GPU est disponible ?  True


In [2]:
!pip install transformers accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer


MODEL_NAME = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,  # Utiliser float16 pour réduire l'empreinte mémoire
    device_map="auto",          # Distribuer sur GPU si disponible, sinon CPU
    trust_remote_code=True      # Nécessaire pour Phi-2
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
AVAILABLE_CONTEXT = {
    # Variables de contexte standard
    "Variables": ["this", "root", "item", "key", "app", "user", "request"],

    # Fonctions standards dans Symfony Expression Language
    "Fonctions": [
        "constant()",
        "defined()",
        "divisible by()",
        "empty()",
        "even()",
        "null()",
        "odd()",
        "same as()"
    ],

    # Opérateurs disponibles
    "Opérateurs": [
        "==", "===", "!=", "!==", "<", ">", "<=", ">=",
        "+", "-", "*", "/", "%",
        "and", "or", "not", "in", "matches", "starts with", "ends with", "contains"
    ],

    # Types pour les tests
    "Types": ["array", "string", "number", "boolean", "null"]
}

# Exemples d'expressions Symfony valides
SYMFONY_EXAMPLES = [
    "userContext.userRole == 'admin'",
    "clientContext.clientId != null",
    "userContext.userEmail matches '/@company\\.com$/'",
    "sessionContext.isActive and userContext.userRole in ['manager', 'admin']"
]

In [12]:
# Fonction améliorée avec retry
def generate_expression(user_intent, available_context=None, examples=None, max_retries=1):
    """
    Génère une suggestion d'expression Symfony avec retry en cas d'échec

    Args:
        user_intent (str): Intention ou question de l'utilisateur
        available_context (dict, optional): Contexte disponible dans l'éditeur
        examples (list, optional): Exemples d'expressions valides
        max_retries (int, optional): Nombre maximal de tentatives en cas d'erreur

    Returns:
        str: Expression Symfony suggérée par le modèle
    """
    # Construction du prompt pour Phi-2
    prompt = "Tu es un assistant spécialisé pour l'éditeur d'expression Symfony (https://symfony.com/doc/current/reference/formats/expression_language.html).\n"
    prompt += "Tu aides les consultants à construire des expressions conditionnelles valides en utilisant la syntaxe Symfony Expression Language.\n\n"

    # Ajout du contexte disponible
    if available_context:
        prompt += "Éléments disponibles dans l'environnement:\n"
        for category, items in available_context.items():
            prompt += f"- {category}: {', '.join(items)}\n"

    # Ajout d'exemples
    if examples:
        prompt += "\nExemples d'expressions Symfony valides:\n"
        for i, example in enumerate(examples):
            prompt += f"{i+1}. {example}\n"

    # Ajouter des instructions spécifiques
    prompt += "\nRègles à suivre:\n"
    prompt += "- Utilise uniquement la syntaxe de Symfony Expression Language\n"
    prompt += "- N'utilise que les variables, contextes et méthodes listés ci-dessus\n"
    prompt += "- Réponds avec une expression simple et directe\n"
    prompt += "- Pour les opérateurs logiques, utilise 'and', 'or', 'not'\n\n"

    # Ajouter la demande de l'utilisateur
    prompt += f"Demande: {user_intent}\n\n"
    prompt += "Expression Symfony: "

    retries = 0
    while retries <= max_retries:
        try:
            # Encoder le prompt
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

            # Génération de la réponse
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.5,
                do_sample=True,
                top_p=0.9,
                eos_token_id=tokenizer.eos_token_id
            )

            # Décodage de la sortie
            full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extraction de la partie pertinente (après "Expression Symfony: ")
            if "Expression Symfony: " in full_response:
                response = full_response.split("Expression Symfony: ")[-1].strip()
            else:
                # Fallback au cas où le modèle ne respecte pas le format
                response = full_response.replace(prompt, "").strip()

            # Nettoyage final (retirer les explications qui pourraient suivre l'expression)
            if "\n" in response:
                response = response.split("\n")[0].strip()

            # Vérification que la réponse n'est pas vide
            if response and not response.isspace() and "```" not in response:
                return response
            else:
                # Si réponse vide ou invalide, retenter
                retries += 1
                print(f"Réponse vide ou invalide, tentative {retries}/{max_retries+1}...")
                # Attendre un peu avant de retenter (pour donner le temps au modèle de se réinitialiser)
                import time
                time.sleep(1)

        except Exception as e:
            # En cas d'erreur, retenter
            retries += 1
            print(f"Erreur lors de la génération: {e}. Tentative {retries}/{max_retries+1}...")
            import time
            time.sleep(1)

    # Si après toutes les tentatives, rien n'a fonctionné
    return "Une erreur est survenue, veuillez réessayer."

In [13]:
user_query = "Je veux vérifier si l'utilisateur a le rôle admin"
result = generate_expression(user_query, AVAILABLE_CONTEXT, SYMFONY_EXAMPLES)
print("Résultat:")
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Résultat:
clientContext.clientId!= null && clientContext.clientConfidence > 0.5


In [14]:
user_query_2 = "Comment vérifier si l'email d'un utilisateur contient @company.com?"
result_2 = generate_expression(user_query_2, AVAILABLE_CONTEXT, SYMFONY_EXAMPLES)
print("Résultat:")
print(result_2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Résultat:
userContext.userEmail contains '/@company\.com$/'


In [15]:
user_query_3 = "Créer une expression qui vérifie que l'utilisateur est actif ET a soit le rôle admin soit le rôle manager"
result_3 = generate_expression(user_query_3, AVAILABLE_CONTEXT, SYMFONY_EXAMPLES)
print("Résultat:")
print(result_3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Résultat:
clientContext.clientId == null and request.parameter.isString()
